In [1]:
import repackage
repackage.up()
import os

from datasets.nuscenes import NuScenesMultipleRadarMultiSweeps
from utils.occupancy import *
from utils.labelling import *
from utils.transforms import *
from utils.postprocessing import *
from utils.ransac_solver import RANSACSolver
from utils.motion_estimation import *
from autolabeler import AutoLabeler
import pandas as pd
import os.path as osp
import numpy as np
import open3d as o3d
from pathlib import Path
from nuscenes import NuScenes
from multiprocessing import cpu_count, Pool, set_start_method
import matplotlib
import matplotlib.pyplot as plt
from utils.visualization import map_pointcloud_to_image, render_pointcloud_in_image, plot_maps
from utils.motion_estimation import remove_dynamic_points
from utils.labelling import get_sps_labels


Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [2]:
DATA_DIR = "/shared/data/nuScenes/"
versions = {'trainval': 'v1.0-trainval', 'test': 'v1.0-test'}
nuscenes_exp = {
    vname: NuScenes(dataroot=DATA_DIR, version=version, verbose=False)
    for vname, version in versions.items()
}

In [3]:
ref_frame = None
NUM_SWEEPS = 5
ref_sensor = None
filter_points = False

DPR_THRESH = 0.15
OCTOMAP_RESOLUTION = 0.15 # For dividing space, for lidar 0.1 is suitable but since radar is sparse a larger value might be better
VOXEL_SIZE = 0.01

DF_PATH = '../sps_nuscenes_more_matches_df.json'
sps_df = pd.read_json(DF_PATH)


ICP_FILTERING = True
SEARCH_IN_RADIUS = True
RADIUS = 1
USE_LIDAR_LABELS = False
USE_OCCUPANCY_PRIORS = True
FILTER_BY_POSES = False
FILTER_BY_RADIUS = False
FILTER_OUT_OF_BOUNDS = False
USE_COMBINED_MAP = True


SENSORS = ["RADAR_FRONT", "RADAR_FRONT_LEFT", "RADAR_FRONT_RIGHT", "RADAR_BACK_LEFT", "RADAR_BACK_RIGHT"]

In [4]:
"""
Given exp dir
Load the labelled maps from there (global frame)
Setup dataloader in sensor frame
Get DPR masks for each sensor PCL
Get DPR masks in global frame
From the loaded labelled map, remove the DPR points --> Save maps_dpr/ (This leaves only auto-labeler points)
From each sensor's PCL:
    - remove the DPR points
    - project a copy to global and retrieve the stable scores from the loaded map post DPR
    - project to local pose
    - concat all sensors
Add this processed frame for the local_dpr_map
Do for all frames and then save the new map --> Save local_maps_dpr/ 
"""

"\nGiven exp dir\nLoad the labelled maps from there (global frame)\nSetup dataloader in sensor frame\nGet DPR masks for each sensor PCL\nGet DPR masks in global frame\nFrom the loaded labelled map, remove the DPR points --> Save maps_dpr/ (This leaves only auto-labeler points)\nFrom each sensor's PCL:\n    - remove the DPR points\n    - project a copy to global and retrieve the stable scores from the loaded map post DPR\n    - project to local pose\n    - concat all sensors\nAdd this processed frame for the local_dpr_map\nDo for all frames and then save the new map --> Save local_maps_dpr/ \n"

In [5]:
LABELLED_MAPS_DIR = '/home/umair/workspace/radar_sps_datasets/nuscenes/maps/'
DPR_MAPS_DIR = '/home/umair/workspace/radar_sps_datasets/nuscenes/maps_dpr/'
LOCAL_DPR_MAPS_DIR = '/home/umair/workspace/radar_sps_datasets/nuscenes/local_maps_dpr/'
[Path(d).mkdir(exist_ok=True, parents=True) for d in [DPR_MAPS_DIR, LOCAL_DPR_MAPS_DIR]]

[None, None]

In [8]:
all_scenes_data = []
for i,row in sps_df.iterrows():
    all_scenes_data.append((row['scene_name'], row['split']))
    for scene, data in row['closest_scenes_data'].items():
        all_scenes_data.append((scene, data['split']))

In [ ]:
scene_name = 'scene-120'
split = 'trainval'
seq = int(scene_name.split('-')[1])

In [ ]:
dataloader = NuScenesMultipleRadarMultiSweeps(
        data_dir=DATA_DIR,
        nusc=nuscenes_exp[split],
        sequence=seq,
        sensors=SENSORS,
        nsweeps=NUM_SWEEPS,
        ref_frame=None,
        ref_sensor=None,
        apply_dpr=False,
        filter_points=False,
        ransac_threshold=-1,
        combine_velocity_components=False
)

In [ ]:
scene_name

In [ ]:
labelled_map = np.loadtxt(os.path.join(LABELLED_MAPS_DIR, f'{scene_name}.asc'), skiprows=1)
labelled_map.shape

In [ ]:
scene_pointclouds = [dataloader[i][0] for i in range(dataloader.num_readings)]
scene_calibs = [dataloader[i][1] for i in range(dataloader.num_readings)]
scene_global_poses = dataloader.global_poses
scene_local_poses = dataloader.local_poses

In [ ]:
ransac_solver = RANSACSolver(threshold=DPR_THRESH, max_iter=10, outdir='output_dpr')
scene_dpr_masks = []
for index, (sensor_pcls, sensor_calibs) in enumerate(zip(scene_pointclouds, scene_calibs)):
    sensor_dpr_masks = {}
    for sensor, pcl, calib in zip(SENSORS, sensor_pcls, sensor_calibs):
        if sensor in ['RADAR_FRONT_LEFT', 'RADAR_FRONT_RIGHT']:
            ## Removing these points since Autoplace does not use these sensors either
            # DPR results get worse if we use them
            sensor_dpr_masks[sensor] = np.zeros(len(pcl)).astype(bool)
            continue
        
        if pcl.shape[0] > 1:
            info = [
                [scene_name, index],
                sensor,
                pcl.shape[0]
            ]
            best_mask, _, _ = ransac_solver.ransac_nusc(info=info, pcl=pcl, vis=(False and sensor == 'RADAR_FRONT'))
            sensor_dpr_masks[sensor] = best_mask
    scene_dpr_masks.append(sensor_dpr_masks)
            # ego_points_sensor = transform_doppler_points(calib, pcl)

In [ ]:
def plot_dpr_masks(scene, map_pcl, mask, size=0.25):
    plt.figure(figsize=(10, 10))
    
    plt.scatter(map_pcl[:, 0], map_pcl[:, 1], s=size, label='static', c='green', alpha=0.5)
    dpr_map = map_pcl[:len(mask)]
    dpr_map = dpr_map[~mask]
    plt.scatter(dpr_map[:, 0], dpr_map[:, 1], s=size, label='dynamic', c='red', alpha=0.5)
    plt.title(scene)
    plt.xlabel('X')
    plt.ylabel('Y')

In [ ]:
sensor_idx = 3
frame_idx = 10
plot_dpr_masks(scene=SENSORS[sensor_idx], map_pcl=scene_pointclouds[frame_idx][sensor_idx], mask=scene_dpr_masks[frame_idx][SENSORS[sensor_idx]])

In [ ]:
local_scene_pcls = []
global_scene_pcls = []
scene_sensor_ids = []

for frame_dpr_masks, frame_pcls, frame_global_pose, frame_local_pose, frame_calibs in zip(scene_dpr_masks, scene_pointclouds, scene_global_poses, scene_local_poses, scene_calibs):
    frame_local_pcl = []
    frame_global_pcl = []
    frame_sensor_ids = []

    for sensor, sensor_pcl, sensor_calib in zip(SENSORS, frame_pcls, frame_calibs):
        dpr_mask = frame_dpr_masks[sensor]
        static_sensor_pcl = sensor_pcl[dpr_mask, :]
        ego_pcl = transform_doppler_points(sensor_calib, static_sensor_pcl)
        ego_local_pcl = transform_doppler_points(frame_local_pose, ego_pcl)
        global_pcl = transform_doppler_points(frame_global_pose, ego_pcl)
        frame_local_pcl.append(ego_local_pcl)
        frame_global_pcl.append(global_pcl)
        frame_sensor_ids.append([sensor] * len(static_sensor_pcl))
    
    local_scene_pcls.append(np.vstack(frame_local_pcl))
    global_scene_pcls.append(np.vstack(frame_global_pcl))
    scene_sensor_ids.append(np.hstack(frame_sensor_ids))

In [ ]:
len(local_scene_pcls), len(global_scene_pcls), len(scene_sensor_ids)

In [ ]:
static_labels = []

for pcl in global_scene_pcls:
    stable_scores = get_sps_labels(labelled_map, pcl)
    static_labels.append(stable_scores)

In [ ]:
len(static_labels)

In [ ]:
static_labels[0]

In [ ]:
scene_global_static_labelled_map = []
scene_local_static_labelled_map = []

for i in range(len(static_labels)):
    local_labelled_scan = np.hstack([local_scene_pcls[i], static_labels[i].reshape(-1 ,1) ])
    global_labelled_scan = np.hstack([global_scene_pcls[i], static_labels[i].reshape(-1 ,1)])

    scene_local_static_labelled_map.append(local_labelled_scan)
    scene_global_static_labelled_map.append(global_labelled_scan)

In [ ]:
scene_local_static_labelled_map = np.vstack(scene_local_static_labelled_map)
scene_global_static_labelled_map = np.vstack(scene_global_static_labelled_map)

scene_local_static_labelled_map.shape, scene_global_static_labelled_map.shape

In [ ]:
plot_maps({
    'with_dynamic_points' : labelled_map,
    'without_dynamic_points' : scene_global_static_labelled_map
}, poses={'with_dynamic_points' : scene_global_poses, 'without_dynamic_points' : scene_global_poses}, zoom_level=-1)

In [ ]:
points = scene_global_static_labelled_map[:, :3]
labels = scene_global_static_labelled_map[:, -1]

mean_point = np.mean(points, axis=0) # color rescaling fix
points = np.vstack([points, mean_point, mean_point + 1])
labels = np.hstack([labels, [0, 1]])

plt.figure(figsize=(10, 10))
plt.scatter(points[:, 0], points[:, 1], c=labels, cmap='RdYlGn', s=0.05)
plt.colorbar(label='Stability')
plt.xlabel('X')
plt.ylabel('Y')
plt.show()

In [ ]:
points = labelled_map[:, :3]
labels = labelled_map[:, -1]

mean_point = np.mean(points, axis=0) # color rescaling fix
points = np.vstack([points, mean_point, mean_point + 1])
labels = np.hstack([labels, [0, 1]])

plt.figure(figsize=(10, 10))
plt.scatter(points[:, 0], points[:, 1], c=labels, cmap='RdYlGn', s=0.05)
plt.colorbar(label='Stability')
plt.xlabel('X')
plt.ylabel('Y')
plt.show()

In [6]:
def get_updated_data(scene_name, dataloader, labelled_map):

    scene_pointclouds = [dataloader[i][0] for i in range(dataloader.num_readings)]
    scene_calibs = [dataloader[i][1] for i in range(dataloader.num_readings)]
    scene_global_poses = dataloader.global_poses
    scene_local_poses = dataloader.local_poses

    scene_label_statistics = {}
    
    ransac_solver = RANSACSolver(threshold=DPR_THRESH, max_iter=10, outdir='output_dpr')
    scene_dpr_masks = []
    for index, (sensor_pcls, sensor_calibs) in enumerate(zip(scene_pointclouds, scene_calibs)):
        sensor_dpr_masks = {}
        for sensor, pcl, calib in zip(SENSORS, sensor_pcls, sensor_calibs):
            if sensor in ['RADAR_FRONT_LEFT', 'RADAR_FRONT_RIGHT']:
                ## Removing these points since Autoplace does not use these sensors either
                # DPR results get worse if we use them
                sensor_dpr_masks[sensor] = np.zeros(len(pcl)).astype(bool)
                continue
            
            if pcl.shape[0] > 1:
                info = [
                    [scene_name, index],
                    sensor,
                    pcl.shape[0]
                ]
                best_mask, _, _ = ransac_solver.ransac_nusc(info=info, pcl=pcl, vis=(False and sensor == 'RADAR_FRONT'))
                sensor_dpr_masks[sensor] = best_mask
        scene_dpr_masks.append(sensor_dpr_masks)
    
    ego_scene_pcls = []
    local_scene_pcls = []
    global_scene_pcls = []
    scene_sensor_ids = []
    merged_scene_dpr_mask = []

    for frame_dpr_masks, frame_pcls, frame_global_pose, frame_local_pose, frame_calibs in zip(scene_dpr_masks, scene_pointclouds, scene_global_poses, scene_local_poses, scene_calibs):
        frame_ego_pcl = []
        frame_local_pcl = []
        frame_global_pcl = []
        frame_sensor_ids = []
        merged_frame_dpr_masks = []

        for sensor, sensor_pcl, sensor_calib in zip(SENSORS, frame_pcls, frame_calibs):
            dpr_mask = frame_dpr_masks[sensor]
            merged_frame_dpr_masks.append(dpr_mask)

            static_sensor_pcl = sensor_pcl[dpr_mask, :]

            num_dynamic_points = len(sensor_pcl) - len(static_sensor_pcl)
            if 'scene_dynamic_points' in scene_label_statistics:
                scene_label_statistics['scene_dynamic_points']+=num_dynamic_points
            else:
                scene_label_statistics['scene_dynamic_points'] = num_dynamic_points

            ego_pcl = transform_doppler_points(sensor_calib, static_sensor_pcl)
            ego_local_pcl = transform_doppler_points(frame_local_pose, ego_pcl)
            global_pcl = transform_doppler_points(frame_global_pose, ego_pcl)
            
            frame_ego_pcl.append(ego_pcl)
            frame_local_pcl.append(ego_local_pcl)
            frame_global_pcl.append(global_pcl)
            frame_sensor_ids.append([sensor] * len(static_sensor_pcl))
        
        ego_scene_pcls.append(np.vstack(frame_ego_pcl))
        local_scene_pcls.append(np.vstack(frame_local_pcl))
        global_scene_pcls.append(np.vstack(frame_global_pcl))
        scene_sensor_ids.append(np.hstack(frame_sensor_ids))
        merged_scene_dpr_mask.append(np.hstack(merged_frame_dpr_masks))

    
    static_labels = []
    for pcl in global_scene_pcls:
        stable_scores = get_sps_labels(labelled_map, pcl)
        static_labels.append(stable_scores)


    scene_global_static_labelled_map = []
    scene_local_static_labelled_map = []

    for i in range(len(static_labels)):
        local_labelled_scan = np.hstack([local_scene_pcls[i], static_labels[i].reshape(-1 ,1) ])
        global_labelled_scan = np.hstack([global_scene_pcls[i], static_labels[i].reshape(-1 ,1)])

        scene_local_static_labelled_map.append(local_labelled_scan)
        scene_global_static_labelled_map.append(global_labelled_scan)

    scene_local_static_labelled_map = np.vstack(scene_local_static_labelled_map)
    scene_global_static_labelled_map = np.vstack(scene_global_static_labelled_map)


    num_unstable_points = np.sum(scene_local_static_labelled_map[:, -1] <= 0.5)
    num_stable_points = np.sum(scene_local_static_labelled_map[:, -1] > 0.5)

    scene_label_statistics['scene_unstable_points'] = num_unstable_points
    scene_label_statistics['scene_stable_points'] = num_stable_points

    updated_data_dict = {
        'labelled_local_static_map' : scene_local_static_labelled_map,
        'labelled_global_static_map' : scene_local_static_labelled_map,

        'static_ego_frames' : ego_scene_pcls,
        'static_frame_labels' : static_labels,

        'scene_label_statistics' : scene_label_statistics,
    }

    return updated_data_dict

    
    

In [9]:
all_scenes_label_statistics = []

for scene_name in os.listdir(LABELLED_MAPS_DIR):
    old_labelled_map = np.loadtxt(os.path.join(LABELLED_MAPS_DIR, scene_name), skiprows=1)
    scene_name = scene_name.replace(".asc", "")
    split = [tpl[1] for tpl in all_scenes_data if tpl[0] == scene_name][-1]
    seq = int(scene_name.split("-")[-1])

    dataloader = NuScenesMultipleRadarMultiSweeps(
        data_dir=DATA_DIR,
        nusc=nuscenes_exp[split],
        sequence=seq,
        sensors=SENSORS,
        nsweeps=NUM_SWEEPS,
        ref_frame=None,
        ref_sensor=None,
        apply_dpr=False,
        filter_points=False,
        ransac_threshold=-1,
        combine_velocity_components=False
    )
    update_data_dict = get_updated_data(scene_name, dataloader, old_labelled_map)

    all_scenes_label_statistics.append({**{'scene_name' : scene_name}, **update_data_dict['scene_label_statistics']})
    
    local_lmap = update_data_dict['labelled_local_static_map']
    global_lmap = update_data_dict['labelled_global_static_map']
    ego_frames = update_data_dict['static_ego_frames']
    scan_stablility_scores = update_data_dict['static_frame_labels']
    
    scans_dir = f'/home/umair/workspace/radar_sps_datasets/nuscenes/sequence/{scene_name}/scans/'
    scans_dpr_dir = f'/home/umair/workspace/radar_sps_datasets/nuscenes/sequence/{scene_name}/scans_dpr/'
    labels_dpr_dir = f'/home/umair/workspace/radar_sps_datasets/nuscenes/sequence/{scene_name}/labels_dpr/'
    

    local_map_file = os.path.join(LOCAL_DPR_MAPS_DIR, f'{scene_name}.asc')
    np.savetxt(local_map_file,
                local_lmap,
                fmt='%.6f', delimiter=' ',
                header='x y z RCS v_x v_y cv_x cv_y stable_prob',
                comments='')
    
    global_map_file = os.path.join(DPR_MAPS_DIR, f'{scene_name}.asc')
    np.savetxt(global_map_file,
                global_lmap,
                fmt='%.6f', delimiter=' ',
                header='x y z RCS v_x v_y cv_x cv_y stable_prob',
                comments='')

    [Path(p).mkdir(parents=True,exist_ok=True) for p in [scans_dpr_dir, labels_dpr_dir]]
    
    scan_files = sorted([file for file in os.listdir(scans_dir)], key=lambda f: float(''.join(filter(lambda x: x.isdigit() or x == '.', f.split('.npy')[0]))))

    for name, scan, label in zip(scan_files, ego_frames, scan_stablility_scores):
        np.save(os.path.join(scans_dpr_dir, f'{name}'), scan)
        np.save(os.path.join(labels_dpr_dir, f'{name}'), label)

    break

In [ ]:
def plot_labelled_map(lmap):
    points = lmap[:, :3]
    labels = lmap[:, -1]

    mean_point = np.mean(points, axis=0) # color rescaling fix
    points = np.vstack([points, mean_point, mean_point + 1])
    labels = np.hstack([labels, [0, 1]])

    plt.figure(figsize=(10, 10))
    plt.scatter(points[:, 0], points[:, 1], c=labels, cmap='RdYlGn', s=0.05)
    plt.colorbar(label='Stability')
    plt.xlabel('X')
    plt.ylabel('Y')
    plt.show()

In [ ]:
plot_labelled_map(update_data_dict['labelled_local_static_map'])

In [ ]:
update_data_dict['scene_label_statistics']